In [2]:
# Aproximación de Simpson compuesta para integral 1D
def trapz1d (n, f, a, b):
    integ = (f(a)+f(b))/2
    h = (b-a)/n
    for i in range(1, n):
        x = a + i*h
        integ += f(x)
    integ *= h
    return integ


# Aproximación de Simpson compuesta para integral 2D
def trapz2d(f, a, b, c, d, n, m):
    integ = 0
    h = (b-a)/n

    # Veamos si c, d son constantes (para áreas de integración rectangulres) 
    # y las convertimos a funciones para que el resto del código funcione
    phi1 = c if callable(c) else lambda x: c
    phi2 = d if callable(d) else lambda x: d

    # En vez de este pedazo, podríamos introducir un if dentro del ciclo y cambiar el coeficiente
    integ = (trapz1d(m, lambda y: f(a, y), phi1(a), phi2(a)) + trapz1d(m, lambda y: f(b, y), phi1(b), phi2(b)))/2
    
    for i in range(1,n-1):
        x = a + i*h
        g = lambda y: f(x,y)
        integ += trapz1d(m, g, phi1(x), phi2(x))
    integ *= h
    return integ


In [3]:
# Aproximación de Simpson compuesta para integral 1D
def simps1d(n, f, a, b):
    integ = f(a)+f(b)   # Valor inicial: primera parte de la suma
    h = (b-a)/n

    # Valores iniciales para el ciclo
    segSum = 0
    terSum = 0
    predX = a # xi anterior

    for i in range(1, n):
        x = a + h*i
        segSum += f(x)
        terSum += f((predX + x)/2)
        predX = x  # Valor para el siguiente ciclo

    # Valores faltantes del ciclo
    terSum += f((predX + b)/2)

    # Coeficientes de las sumas
    segSum *= 2
    terSum *= 4

    # Agregamos valores a aprox y multiplicamos por factor
    integ += segSum + terSum
    integ *= h/6

    return integ


# Aproximación de Simpson compuesta para integral 2D
def simps2d(f, a, b, c, d, n, m):
    # Veamos si c, d son constantes (para áreas de integración rectangulres) 
    # y las convertimos a funciones para que el resto del código funcione
    phi1 = c if callable(c) else lambda x: c
    phi2 = d if callable(d) else lambda x: d 

    # Integral interior como función de x
    F = lambda x: simps1d(m, lambda y: f(x, y), phi1(x), phi2(x))

    integ = F(a)+F(b)   # Valor inicial: primera parte de la suma
    h = (b-a)/n

    # Valores iniciales para el ciclo
    segSum = 0
    terSum = 0
    predX = a
    for i in range(1, n):
        x = a + i*h
        segSum += F(x)
        terSum += F((predX+x)/2)
        predX = x # Valor para el siguiente ciclo

    # Valores faltantes del ciclo
    terSum += F((predX+b)/2)

    # Coeficientes de las sumas
    segSum *= 2
    terSum *= 4

    # Agregamos valores a aprox y multiplicamos por factor
    integ += segSum + terSum
    integ *= h/6

    return integ

In [4]:
import numpy as np

# Aproximación de Gauss compuesta para integral 1D
# k es número de abscisas
def gauss1d(f, a, b, n, k):
    h = (b-a)/n
    x, w  = np.polynomial.legendre.leggauss(k)  # Pesos y abscisas
    integ = 0.0

    # Ciclo que itera intervalos
    for i in range(n):
        xa = a + i*h
        xb = xa + h
        T = lambda u: (xa + xb + u*h)/2     # Transformación para intervalo [-1,1]
        # Integración gaussiana para cada intervalo
        for l in range(k):
            integ += w[l] * f(T( x[l] ))
    integ *= h/2
    return integ

def gauss2d(f, a, b, c, d, n, m, kn, km):
    # Veamos si c, d son constantes (para áreas de integración rectangulres) 
    # y las convertimos a funciones para que el resto del código funcione
    phi1 = c if callable(c) else lambda x: c
    phi2 = d if callable(d) else lambda x: d 

    F = lambda x: gauss1d(lambda y: f(x,y), phi1(x), phi2(x), m, km)

    h = (b-a)/n
    x, w = np.polynomial.legendre.leggauss(kn)
    integ = 0.0

    # Ciclo que itera intervalos
    for i in range(n):
        xa = a + h*i
        xb = xa + h
        T = lambda u: (xa + xb + u*h)/2 # Transformación para intervalo [-1,1]
        # Integración gaussiana para cada intervalo
        for l in range(kn):
            integ += w[l] * F(T( x[l] ))
    integ *= h/2
    return integ


In [5]:
import numpy as np

# Funciones a evlauar
funcs = {
    "seno": lambda x,y: np.sin(x*y), 
    "x^2 + y^2": lambda x,y: x**2 + y**2, 
    "x^5 + y^5": lambda x,y: x**5 + y**5
}

# Valor real de la doble integral (según WolframAlpha)
valoresReales = [
    0.4249603946480658226404103877736619464629130316183373589717236607, 
    25*(np.pi**4)/48, 
    139*(np.pi**7)/336
]

# Definimos los límites de integración para la variable x
a = 0.0
b = np.pi

# Definimos el número de subintervalos en cada dirección
n = 10
m = 10

# Definimos las funciones phi1 y phi2 como funciones que dependen de x
def phi1(x):
    return x/2

def phi2(x):
    return 3*x/2


In [7]:
import inspect

minExp = 5
maxExp = 8
abscisas = [2,4]

for i in range(len(funcs)):
  nomFunc = list(funcs.keys())[i]
  f = funcs[nomFunc]
  valorReal = valoresReales[i]

  print("="*60+"\n")
  print("Aproximación de la integral en dos dimensiones para la función "+nomFunc+", en el intervalo [0,pi]")
  print("Valor real: ", valorReal)
  print("")


  for i in range(minExp, maxExp+1):
      n = 2**i
      m = 2**i
      integral_trapz2d = trapz2d(f, a, b, phi1, phi2, n, m)
      errorTrapz2d = abs(valorReal-integral_trapz2d)
      integral_simp2d = simps2d(f, a, b, phi1, phi2, n, m)
      errorSimps2d = abs(valorReal-integral_simp2d)
      print("Número de intervalos: ", " ","n="+str(n)+", m="+str(m))
      print("Trapecio:             ", integral_trapz2d, " ", "error abs: ", errorTrapz2d, " ", "error rel: ", errorTrapz2d/valorReal)
      print("Simpson:              ", integral_simp2d, " ", "error abs: ", errorSimps2d, " ", "error rel: ", errorSimps2d/valorReal)
      for numAbs in abscisas:
        integral_gauss2d = gauss2d(f, a, b, phi1, phi2, n, m, numAbs, numAbs)
        errorGauss2d = abs(valorReal-integral_gauss2d)
        print("Gauss con "+ str(numAbs) +" abscisas: ", integral_gauss2d, " ", "error abs: ", errorGauss2d, " ", "error rel: ", errorGauss2d/valorReal)
      print("")
  print("="*60)


Aproximación de la integral en dos dimensiones para la función seno, en el intervalo [0,pi]
Valor real:  0.42496039464806584

Número de intervalos:    n=32, m=32
Trapecio:              0.43899040373960224   error abs:  0.0140300090915364   error rel:  0.03301486272186719
Simpson:               0.4249527218703657   error abs:  7.672777700140632e-06   error rel:  1.8055277142932107e-05
Gauss con 2 abscisas:  0.4249655187602103   error abs:  5.124112144483384e-06   error rel:  1.205785811811229e-05
Gauss con 4 abscisas:  0.4249603946590529   error abs:  1.0987044607446705e-11   error rel:  2.5854279000624775e-11

Número de intervalos:    n=64, m=64
Trapecio:              0.42151181420610584   error abs:  0.003448580441960003   error rel:  0.008115063157393692
Simpson:               0.4249599245687045   error abs:  4.7007936132548167e-07   error rel:  1.1061721686200463e-06
Gauss con 2 abscisas:  0.4249607081709314   error abs:  3.135228655581912e-07   error rel:  7.377696121960672e-07
Ga